# Dr. Kim Morita: Preventive Oral Care Expert, Morita Dentistry

### “Did you know that a healthy mouth can protect you from far more than tooth decay? Diabetes, heart disease, stroke, and a multitude of other diseases and medical conditions have been linked to dental disease. As such, balancing the mouth’s bacterial ecosystem is a great way to stop cavities, stop gum disease, and improve overall health.” —via [Stanford BeWell](https://events.stanford.edu/event/dental_wellness_2617)

### With Dr. Morita, we examine an AI-enhanced patient experience challenge with an automation to use for her morning huddles. 

As Dr. Morita's practice has grown, it becomes more complex and difficult to achieve personalized care at scale. Not only is there a need to know the patient's preferred scheduling for times and days, but also there are often appointment specific preferences like:

* warm water during cleaning
* blankets or pillow
* whether the patient needs premedication
* fluoride/no fluorida
* anesthesia preferences

Dr. Morita's office does morning huddles to help highlight a handful of notable cases for the day's schedule. It can be a lot of information for the team to process the volume of details. How can she simplify the process and yet create highly customized schedules?

## 🔥 Let's get the required packages and fire up a kernel

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"

In [ ]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🧑‍🍳 Let's get cooking on this problem together!

### 🔥 We fire up Semantic Kernel's Experimental Agents ...

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: YamlDotNet, 13.7.1"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

public class Agenty
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

List<string> agentNames = ["MoritaHuddleManager"];
string team = "";
int i = 1;
List<(string Name, string Instructions)> agentInfo = new();

foreach(var a in agentNames)
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();

    var p = deserializer.Deserialize<Agenty>(yaml);
    Console.WriteLine($"Agent: {a}");
    team += $"{i}) {p.Name}: {p.Description}, ";
    i++;
    agentInfo.Add((p.Name, p.Instructions));
}

agentInfo

### 🐣 Let's hatch the agent(s)

In [ ]:
#pragma warning disable SKEXP0101

static readonly List<IAgent> s_agents = new();

async Task<IAgent> CreateAgentAsync(string name, string jobDescription)
{
    return Track(
        await new AgentBuilder()
            .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
            .WithInstructions(jobDescription)
            .WithName(name)
            .WithDescription(name)
            .BuildAsync());
}

IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

var agents = new List<IAgent>();
foreach (var (name, jobDescription) in agentInfo)
{
    var agent = await CreateAgentAsync(name, jobDescription);

    agents.Add(agent);
}

agents

### 🧑‍🍳 Let's grab the important info for today

Note that we'll intentionally include a wrong name in the info below to see what happens.

In [ ]:
static string patientsToday = @"The special requirements for patients coming today are as follows:

1. Jim's patient at 9AM has a back problem and will need pillows and a blanket
2. Jan's patient at 10AM needs warm water in the dental unit tank due to teeth sensitivity
";

### 🧵 We need a thread

In [ ]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await agents[0].NewThreadAsync();
await thread.AddUserMessageAsync($"{patientsToday}");

### 🕵️💨 And then we're ready to let the agent do its thing

In [ ]:
#pragma warning disable SKEXP0101

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🎤 Voice makes it easier to work with

In [ ]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/Utils.cs 
#!import ../config/AzureSpeech.cs

Utils.LoadEnvFile();

string subscriptionKey = Environment.GetEnvironmentVariable("AZURE_SPEECH_KEY");
string subscriptionRegion = Environment.GetEnvironmentVariable("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

### 👂🕵️ The agent can lend their ear

In [ ]:
#pragma warning disable SKEXP0101

Console.WriteLine("🎤 As a huddle member, say something to the agent ...");
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"You: {recognizedText}");

//string recognizedText = "I'm Jim, please tell me should I do with my patient today?";

await thread.AddUserMessageAsync($"{recognizedText}. Respond concisely as just a text message and not as in a letter.");

var agentMessages = await thread.InvokeAsync(agents.Last()).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 📣🗣️ And we can let the agent speak up

In [ ]:
#pragma warning disable SKEXP0101

await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

### 📦 With all the special instructions ready, they can be emailed to all

In [ ]:
#pragma warning disable SKEXP0101

// give the AI a plugin for email here
// and just ask it to send the emails

### 🧼 We'll close with some cleanup

In [ ]:
#pragma warning disable SKEXP0101

Console.WriteLine("🧽 Cleaning up ...");

if (thread != null)
{
    Console.WriteLine("Thread going away ...");
    thread.DeleteAsync();
}

if (s_agents.Any())
{
    Console.WriteLine("Agents going away ...");
    // delete all agents
    // loop through the agents and delete them
    foreach (var agent in s_agents)
    {
        await agent.DeleteAsync();
    }
    s_agents.Clear();
}